### Imports

In [1]:
import torch
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import os


/Users/ibraheemalsaghier/anaconda3/envs/personalwebsite/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load in Data 

In [6]:
df = pd.read_csv("./Datasets/combined_features.csv")
 

### Normalize Data

### Remove Redundant Aspects 

In [7]:
df.head(10)

,userId,movieId,rating,timestamp,imdbId,total_ratings,rating_avg,rating_std,rating_proportions,bucket_0_2_count,...,archive_sound,casting_director,cinematographer,composer,director,editor,producer,production_designer,self,writer
0,1,17,4.0,944249077,114388,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,25,1.0,944250228,113627,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,29,2.0,943230976,112682,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,30,5.0,944249077,115012,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,32,5.0,943228858,114746,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,34,2.0,943228491,112431,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,36,1.0,944249008,112818,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,80,5.0,944248943,112445,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,110,3.0,943231119,112573,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,111,5.0,944249008,75314,141,3.531915,1.53787,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'imdbId', 'total_ratings',
       'rating_avg', 'rating_std', 'rating_proportions', 'bucket_0_2_count',
       'bucket_0_2_proportion', 'bucket_3_4_count', 'bucket_3_4_proportion',
       'bucket_5_6_count', 'bucket_5_6_proportion', 'bucket_7_8_count',
       'bucket_7_8_proportion', 'bucket_9_10_count', 'bucket_9_10_proportion',
       'ratio_bucket_0_2_to_bucket_3_4', 'ratio_bucket_0_2_to_bucket_5_6',
       'ratio_bucket_0_2_to_bucket_7_8', 'ratio_bucket_0_2_to_bucket_9_10',
       'ratio_bucket_3_4_to_bucket_5_6', 'ratio_bucket_3_4_to_bucket_7_8',
       'ratio_bucket_3_4_to_bucket_9_10', 'ratio_bucket_5_6_to_bucket_7_8',
       'ratio_bucket_5_6_to_bucket_9_10', 'ratio_bucket_7_8_to_bucket_9_10',
       'high_vs_low_ratio', 'primaryTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'directors',
       'writers', 'actor', 'actress', 'archive_footage', 'archive_sound',
       'casting_dir

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31972560 entries, 0 to 31972559
Data columns (total 52 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   userId                           int64  
 1   movieId                          int64  
 2   rating                           float64
 3   timestamp                        int64  
 4   imdbId                           int64  
 5   total_ratings                    int64  
 6   rating_avg                       float64
 7   rating_std                       float64
 8   rating_proportions               object 
 9   bucket_0_2_count                 int64  
 10  bucket_0_2_proportion            float64
 11  bucket_3_4_count                 int64  
 12  bucket_3_4_proportion            float64
 13  bucket_5_6_count                 int64  
 14  bucket_5_6_proportion            float64
 15  bucket_7_8_count                 int64  
 16  bucket_7_8_proportion            float64
 17  bucket

In [2]:
import sys
print(f"Python path: {sys.executable}")
print("\nInstalled packages:")
!pip list | grep matplotlib

Python path: /Users/ibraheemalsaghier/anaconda3/envs/personalwebsite/bin/python

Installed packages:
matplotlib-inline  0.1.7
